In [1]:
##工具包导入
import pandas as pd
import numpy as np
from fbprophet import Prophet
import pandas as pd
import numpy as np
import lightgbm as lgb
from tqdm import tqdm,tqdm_notebook, tnrange
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings("ignore")

The text.latex.unicode rcparam was deprecated in Matplotlib 3.0 and will be removed in 3.2.
The savefig.frameon rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
The pgf.debug rcparam was deprecated in Matplotlib 3.0 and will be removed in 3.2.
The verbose.level rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
The verbose.fileo rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.


# 数据读取及预处理部分

In [2]:
#数据路径
train_path = './train/'
test_path = './test/'

train_v2 = pd.read_csv(train_path + 'train_v2.csv')
wkd_v1 = pd.read_csv(train_path + 'wkd_v1.csv')

test_v2_day = pd.read_csv(test_path + 'test_v2_day.csv')##要预测每天业务量
test_v2_periods = pd.read_csv(test_path + 'test_v2_periods.csv')##预测各岗位每天每半小时粒度的业务总量

all_wash = pd.merge(wkd_v1, train_v2, how='inner', on=['date'])
#计算不同岗位每日工作量
all_amount = all_wash.groupby(['date','post_id'])['amount'].sum().reset_index()
A_amount = all_amount.loc[all_amount['post_id'] == 'A']
B_amount = all_amount.loc[all_amount['post_id'] == 'B']
#计算不同岗位不同时间段工作量
all_periods = all_wash.groupby(['date','post_id','periods'])['amount'].sum().reset_index()
A_periods = all_periods.loc[all_periods['post_id'] == 'A']
B_periods = all_periods.loc[all_periods['post_id'] == 'B']

#对不同时间段工作量在时间段18-37以外的amount设置为0
def ts(periods, amount):
    if (periods <= 17) | (periods >= 38):
        amount = 0
    return amount

A_periods['amount'] = A_periods.apply(lambda x: ts(x['periods'], x['amount']), axis=1)
B_periods['amount'] = B_periods.apply(lambda x: ts(x['periods'], x['amount']), axis=1)

# 任务一：预测每天业务量

In [3]:
##用上面得到的最优参数来运行
A_amount_params ={
    'growth': 'linear',
     'n_changepoints': 25,
     'changepoint_range': 0.9,
     'changepoint_prior_scale': 0.1,
     'seasonality_mode': 'additive',
     'seasonality_prior_scale': 5}
B_amount_params ={
    'growth': 'linear',
    'n_changepoints': 25,
    'changepoint_range': 0.9,
    'changepoint_prior_scale': 0.2,
    'seasonality_prior_scale': 10,
    'seasonality_mode': 'multiplicative'}

In [4]:
##预测岗位A的每天业务量
#Prophet 所需要的两列名称是 'ds' 和 'y'
A_amount = A_amount.rename(columns={'date':'ds', 'amount':'y'})
#时间修改成 YYYY-MM-DD hh:mm:ss 的形式
A_amount['ds'] = pd.to_datetime(A_amount['ds'], format='%Y-%m-%d %H:%M:%S')
#初始化模型：
A_amount_m = Prophet(**A_amount_params)
#拟合模型：
A_amount_m.fit(A_amount)
#计算预测值：periods 表示需要预测的点数，freq 表示时间序列的频率。这里需要预测30天
A_amount_future = A_amount_m.make_future_dataframe(periods=31, freq='D')
A_amount_forecast = A_amount_m.predict(A_amount_future)
#选对应目标时间段，天数是2020-11-1->2020-11-30
A_amount_predit = A_amount_forecast.loc[(A_amount_forecast['ds']>='2020-12-1')&(A_amount_forecast['ds']<='2020-12-31')]
A_amount_predit = A_amount_predit[['ds', 'yhat']]
A_amount_predit = A_amount_predit.rename(columns={'ds':'date','yhat':'amount'})
A_amount_predit['post_id'] = 'A' 
A_amount_predit['amount'] = A_amount_predit['amount'].astype(int)
#选对应列
A_amount_predit = A_amount_predit.loc[:,['date','post_id','amount']]
#修改日期格式
A_amount_predit.reset_index(inplace=True, drop=True)
A_amount_predit['date'] = A_amount_predit['date'].astype(str).str.replace('-','/')
A_amount_predit['date'] = A_amount_predit['date'].astype(str).str.replace('/01','/1')
A_amount_predit['date'] = A_amount_predit['date'].astype(str).str.replace('/02','/2')
A_amount_predit['date'] = A_amount_predit['date'].astype(str).str.replace('/03','/3')
A_amount_predit['date'] = A_amount_predit['date'].astype(str).str.replace('/04','/4')
A_amount_predit['date'] = A_amount_predit['date'].astype(str).str.replace('/05','/5')
A_amount_predit['date'] = A_amount_predit['date'].astype(str).str.replace('/06','/6')
A_amount_predit['date'] = A_amount_predit['date'].astype(str).str.replace('/07','/7')
A_amount_predit['date'] = A_amount_predit['date'].astype(str).str.replace('/08','/8')
A_amount_predit['date'] = A_amount_predit['date'].astype(str).str.replace('/09','/9')
print("A_amount is OK")

##预测岗位B的每天业务量
#Prophet 所需要的两列名称是 'ds' 和 'y'
B_amount = B_amount.rename(columns={'date':'ds', 'amount':'y'})
#时间修改成 YYYY-MM-DD hh:mm:ss 的形式
B_amount['ds'] = pd.to_datetime(B_amount['ds'], format='%Y-%m-%d %H:%M:%S')
#初始化模型：
B_amount_m = Prophet(**B_amount_params)
#拟合模型：
B_amount_m.fit(B_amount)
#计算预测值：periods 表示需要预测的点数，freq 表示时间序列的频率。这里需要预测30天
B_amount_future = B_amount_m.make_future_dataframe(periods=31, freq='B')
B_amount_forecast = B_amount_m.predict(B_amount_future)
#选对应目标时间段，天数是2020-11-1->2020-11-30
B_amount_predit = B_amount_forecast.loc[(B_amount_forecast['ds']>='2020-12-1')&(B_amount_forecast['ds']<='2020-12-31')]
B_amount_predit = B_amount_predit[['ds', 'yhat']]
B_amount_predit = B_amount_predit.rename(columns={'ds':'date','yhat':'amount'})
B_amount_predit['post_id'] = 'B' 
B_amount_predit['amount'] = B_amount_predit['amount'].astype(int)
#选对应列
B_amount_predit = B_amount_predit.loc[:,['date','post_id','amount']]
#修改日期格式
B_amount_predit.reset_index(inplace=True, drop=True)
B_amount_predit['date'] = B_amount_predit['date'].astype(str).str.replace('-','/')
B_amount_predit['date'] = B_amount_predit['date'].astype(str).str.replace('/01','/1')
B_amount_predit['date'] = B_amount_predit['date'].astype(str).str.replace('/02','/2')
B_amount_predit['date'] = B_amount_predit['date'].astype(str).str.replace('/03','/3')
B_amount_predit['date'] = B_amount_predit['date'].astype(str).str.replace('/04','/4')
B_amount_predit['date'] = B_amount_predit['date'].astype(str).str.replace('/05','/5')
B_amount_predit['date'] = B_amount_predit['date'].astype(str).str.replace('/06','/6')
B_amount_predit['date'] = B_amount_predit['date'].astype(str).str.replace('/07','/7')
B_amount_predit['date'] = B_amount_predit['date'].astype(str).str.replace('/08','/8')
B_amount_predit['date'] = B_amount_predit['date'].astype(str).str.replace('/09','/9')
print("B_amount is OK")

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


A_amount is OK


INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


B_amount is OK


In [5]:
result = pd.merge(test_v2_day, A_amount_predit,how ='left', on=['date','post_id'])
result = result.fillna(0)
result['amount'] = result['amount_x'] + result['amount_y']
result = result.loc[:,['date','post_id','amount']]
result['amount'] = result['amount'].astype(int)
result = pd.merge(result, B_amount_predit,how ='left', on=['date','post_id'])
result = result.fillna(0)
result['amount'] = result['amount_x'] + result['amount_y']
result = result.loc[:,['date','post_id','amount']]
result['amount'] = result['amount'].astype(int)

In [6]:
##记得在最终的txt文件第一行加上‘date,post_id,amount’，不然行数不对
result.to_csv('./result/test_B_day_20210511_1.txt', index=False, header=False, sep=',')
result

,date,post_id,amount
0,2020/12/1,A,22657
1,2020/12/1,B,3119
2,2020/12/2,A,22050
3,2020/12/2,B,3214
4,2020/12/3,A,21642
5,2020/12/3,B,3173
6,2020/12/4,A,22350
7,2020/12/4,B,3446
8,2020/12/5,A,12951
9,2020/12/5,B,0


# 任务二：预测每天半小时业务量

In [7]:
# 预测A岗位
#将A岗位以每小时为粒度分开为两个表
A_periods_time1 = A_periods[A_periods.index%2==0]
A_periods_time2 = A_periods[A_periods.index%2==1]
#将B岗位以每小时为粒度分开为两个表
B_periods_time1 = B_periods[B_periods.index%2==0]
B_periods_time2 = B_periods[B_periods.index%2==1]
#将data和period合并转换成时间段
time = {1:'00:30:00',
        2:'01:00:00',
        3:'01:30:00',
        4:'02:00:00',
        5:'02:30:00',
        6:'03:00:00',
        7:'03:30:00',
        8:'04:00:00',
        9:'04:30:00',
        10:'05:00:00',
        11:'05:30:00',
        12:'06:00:00',
        13:'06:30:00',
        14:'07:00:00',
        15:'07:30:00',
        16:'08:00:00',
        17:'08:30:00',
        18:'09:00:00',
        19:'09:30:00',
        20:'10:00:00',
        21:'10:30:00',
        22:'11:00:00',
        23:'11:30:00',
        24:'12:00:00',
        25:'12:30:00',
        26:'13:00:00',
        27:'13:30:00',
        28:'14:00:00',
        29:'14:30:00',
        30:'15:00:00',
        31:'15:30:00',
        32:'16:00:00',
        33:'16:30:00',
        34:'17:00:00',
        35:'17:30:00',
        36:'18:00:00',
        37:'18:30:00',
        38:'19:00:00',
        39:'19:30:00',
        40:'20:00:00',
        41:'20:30:00',
        42:'21:00:00',
        43:'21:30:00',
        44:'22:00:00',
        45:'22:30:00',
        46:'23:00:00',
        47:'23:30:00',
        48:'00:00:00'
       }

A_periods_time1['time'] = A_periods_time1['periods'].replace(time)
A_periods_time1['date'] = pd.to_datetime(A_periods_time1.date) + pd.to_timedelta(A_periods_time1.time, unit='h')
A_periods_time1 = A_periods_time1.loc[:, ['date', 'amount']]
A_periods_time2['time'] = A_periods_time2['periods'].replace(time)
A_periods_time2['date'] = pd.to_datetime(A_periods_time2.date) + pd.to_timedelta(A_periods_time2.time, unit='h')
A_periods_time2 = A_periods_time2.loc[:, ['date', 'amount']]

B_periods_time1['time'] = B_periods_time1['periods'].replace(time)
B_periods_time1['date'] = pd.to_datetime(B_periods_time1.date) + pd.to_timedelta(B_periods_time1.time, unit='h')
B_periods_time1 = B_periods_time1.loc[:, ['date', 'amount']]
B_periods_time2['time'] = B_periods_time2['periods'].replace(time)
B_periods_time2['date'] = pd.to_datetime(B_periods_time2.date) + pd.to_timedelta(B_periods_time2.time, unit='h')
B_periods_time2 = B_periods_time2.loc[:, ['date', 'amount']]

# 使用最优参数进行预测

In [8]:
#使用上面得到的最优参数
A_periods_time1_params = {
    'growth': 'linear', 
    'n_changepoints': 25, 
    'changepoint_range': 0.8, 
    'changepoint_prior_scale': 10, 
    'seasonality_mode': 'multiplicative', 
    'seasonality_prior_scale': 10}
A_periods_time2_params ={
    'growth': 'linear',
     'n_changepoints': 25,
     'changepoint_range': 0.8,
     'changepoint_prior_scale': 0.05,
     'seasonality_mode': 'multiplicative',
     'seasonality_prior_scale': 10}
B_periods_time_params ={
    'growth': 'linear', 
    'n_changepoints': 35, 
    'changepoint_range': 0.9, 
    'changepoint_prior_scale': 0.1, 
    'seasonality_mode': 'multiplicative', 
    'seasonality_prior_scale': 5,
    'holidays_prior_scale': 15}

In [9]:
print("wait almost three minutes")
##(time1)
##预测岗位A的每天每小时粒度业务量
#Prophet 所需要的两列名称是 'ds' 和 'y'
A_periods_time1 =A_periods_time1.rename(columns={'date':'ds', 'amount':'y'})
#时间修改成 YYYY-MM-DD hh:mm:ss 的形式
A_periods_time1['ds'] = pd.to_datetime(A_periods_time1['ds'], format='%Y-%m-%d %H:%M:%S')

#初始化模型：
A_periods_time1_m = Prophet(**A_periods_time1_params)
#拟合模型：
A_periods_time1_m.fit(A_periods_time1)
#计算预测值：periods 表示需要预测的点数，freq 表示时间序列的频率。(一定要查表对应上)
A_periods_time1_future = A_periods_time1_m.make_future_dataframe(periods=750, freq='H')
A_periods_time1_forecast = A_periods_time1_m.predict(A_periods_time1_future)
#选时间段，因为是小时，所以要小于等于2020-12-1
A_periods_time1 = A_periods_time1_forecast.loc[(A_periods_time1_forecast['ds']>='2020-11-1')&(A_periods_time1_forecast['ds']<='2020-12-1')]
A_periods_time1 = A_periods_time1[['ds', 'yhat']]
A_periods_time1 =A_periods_time1.rename(columns={'ds':'date','yhat':'amount'})
A_periods_time1['post_id'] = 'A' 
#数据转化成int类型
A_periods_time1['amount'] = A_periods_time1['amount'].astype(int)
#选对应列
A_periods_time1 = A_periods_time1.loc[:,['date','post_id','amount']]
print("A_time1 is OK")

##(time2)
##预测岗位A的每天每小时粒度业务量
#Prophet 所需要的两列名称是 'ds' 和 'y'
A_periods_time2 =A_periods_time2.rename(columns={'date':'ds', 'amount':'y'})
#时间修改成 YYYY-MM-DD hh:mm:ss 的形式
A_periods_time2['ds'] = pd.to_datetime(A_periods_time2['ds'], format='%Y-%m-%d %H:%M:%S')
#初始化模型：
A_periods_time2_m = Prophet(**A_periods_time2_params)
#拟合模型：
A_periods_time2_m.fit(A_periods_time2)
#计算预测值：periods 表示需要预测的点数，freq 表示时间序列的频率。(一定要查表对应上)
A_periods_time2_future = A_periods_time2_m.make_future_dataframe(periods=750, freq='H')
A_periods_time2_forecast = A_periods_time2_m.predict(A_periods_time2_future)
#选时间段
A_periods_time2 = A_periods_time2_forecast.loc[(A_periods_time2_forecast['ds']>='2020-11-1')&(A_periods_time2_forecast['ds']<'2020-12-1')]
A_periods_time2 = A_periods_time2[['ds', 'yhat']]
A_periods_time2 =A_periods_time2.rename(columns={'ds':'date','yhat':'amount'})
A_periods_time2['post_id'] = 'A' 
#数据转化成int类型
A_periods_time2['amount'] = A_periods_time2['amount'].astype(int)
#选对应列
A_periods_time2 = A_periods_time2.loc[:,['date','post_id','amount']]
print("A_time2 is OK")


wait almost three minutes
A_time1 is OK
A_time2 is OK


In [10]:
print("wait almost three minutes")
##(time1)
##预测岗位B的每天每小时粒度业务量
#Prophet 所需要的两列名称是 'ds' 和 'y'
B_periods_time1 =B_periods_time1.rename(columns={'date':'ds', 'amount':'y'})
#时间修改成 YYYY-MM-DD hh:mm:ss 的形式
B_periods_time1['ds'] = pd.to_datetime(B_periods_time1['ds'], format='%Y-%m-%d %H:%M:%S')
#初始化模型：
B_periods_time1_m = Prophet(**B_periods_time_params)
#拟合模型：
B_periods_time1_m.fit(B_periods_time1)
#计算预测值：periods 表示需要预测的点数，freq 表示时间序列的频率。(一定要查表对应上)
B_periods_time1_future = B_periods_time1_m.make_future_dataframe(periods=750, freq='H')
B_periods_time1_forecast = B_periods_time1_m.predict(B_periods_time1_future)
#选时间段
B_periods_time1 = B_periods_time1_forecast.loc[(B_periods_time1_forecast['ds']>='2020-11-1')&(B_periods_time1_forecast['ds']<='2020-12-1')]
B_periods_time1 = B_periods_time1[['ds', 'yhat']]
B_periods_time1 = B_periods_time1.rename(columns={'ds':'date','yhat':'amount'})
B_periods_time1['post_id'] = 'B' 
#数据转化成int类型
B_periods_time1['amount'] = B_periods_time1['amount'].astype(int)
#选对应列
B_periods_time1 = B_periods_time1.loc[:,['date','post_id','amount']]
print("B_time1 is OK")

##(time2)
##预测岗位B的每天每小时粒度业务量
#Prophet 所需要的两列名称是 'ds' 和 'y'
B_periods_time2 =B_periods_time2.rename(columns={'date':'ds', 'amount':'y'})
#时间修改成 YYYY-MM-DD hh:mm:ss 的形式
B_periods_time2['ds'] = pd.to_datetime(B_periods_time2['ds'], format='%Y-%m-%d %H:%M:%S')
#初始化模型：
B_periods_time2_m = Prophet(**B_periods_time_params)
#拟合模型：
B_periods_time2_m.fit(B_periods_time2)
#计算预测值：periods 表示需要预测的点数，freq 表示时间序列的频率。(一定要查表对应上)
B_periods_time2_future = B_periods_time2_m.make_future_dataframe(periods=750, freq='H')
B_periods_time2_forecast = B_periods_time2_m.predict(B_periods_time2_future)
#选时间段
B_periods_time2 = B_periods_time2_forecast.loc[(B_periods_time2_forecast['ds']>='2020-11-1')&(B_periods_time2_forecast['ds']<'2020-12-1')]
B_periods_time2 = B_periods_time2[['ds', 'yhat']]
B_periods_time2 =B_periods_time2.rename(columns={'ds':'date','yhat':'amount'})
B_periods_time2['post_id'] = 'B' 
#数据转化成int类型
B_periods_time2['amount'] = B_periods_time2['amount'].astype(int)
#选对应列
B_periods_time2 = B_periods_time2.loc[:,['date','post_id','amount']]
print("B_time2 is OK")

wait almost three minutes
B_time1 is OK
B_time2 is OK


# 转换数据的格式

In [12]:
#将时间段转回去
num  = {'00:30:00':1,
        '01:00:00':2,
        '01:30:00':3,
        '02:00:00':4,
        '02:30:00':5,
        '03:00:00':6,
        '03:30:00':7,
        '04:00:00':8,
        '04:30:00':9,
        '05:00:00':10,
        '05:30:00':11,
        '06:00:00':12,
        '06:30:00':13,
        '07:00:00':14,
        '07:30:00':15,
        '08:00:00':16,
        '08:30:00':17,
        '09:00:00':18,
        '09:30:00':19,
        '10:00:00':20,
        '10:30:00':21,
        '11:00:00':22,
        '11:30:00':23,
        '12:00:00':24,
        '12:30:00':25,
        '13:00:00':26,
        '13:30:00':27,
        '14:00:00':28,
        '14:30:00':29,
        '15:00:00':30,
        '15:30:00':31,
        '16:00:00':32,
        '16:30:00':33,
        '17:00:00':34,
        '17:30:00':35,
        '18:00:00':36,
        '18:30:00':37,
        '19:00:00':38,
        '19:30:00':39,
        '20:00:00':40,
        '20:30:00':41,
        '21:00:00':42,
        '21:30:00':43,
        '22:00:00':44,
        '22:30:00':45,
        '23:00:00':46,
        '23:30:00':47,
        '00:00:00':48
       }

#拆分日期和时间
temp1 = pd.DatetimeIndex(A_periods_time1['date'])
A_periods_time1['date'] = temp1.date
A_periods_time1['time'] = temp1.time
#替换会时间段
A_periods_time1.reset_index(inplace=True, drop=True)
A_periods_time1['periods'] = A_periods_time1['time'].astype(str).replace(num)
#将最后的A需要的数据输出来
A_periods_time1 = A_periods_time1.loc[:, ['date', 'post_id','periods','amount']]

#拆分日期和时间
temp2 = pd.DatetimeIndex(A_periods_time2['date'])
A_periods_time2['date'] = temp2.date
A_periods_time2['time'] = temp2.time
#替换会时间段
A_periods_time2.reset_index(inplace=True, drop=True)
A_periods_time2['periods'] = A_periods_time2['time'].astype(str).replace(num)
#将最后的A需要的数据输出来
A_periods_time2 = A_periods_time2.loc[:, ['date', 'post_id','periods','amount']]

#拆分日期和时间
temp3 = pd.DatetimeIndex(B_periods_time1['date'])
B_periods_time1['date'] = temp3.date
B_periods_time1['time'] = temp3.time
#替换会时间段
B_periods_time1.reset_index(inplace=True, drop=True)
B_periods_time1['periods'] = B_periods_time1['time'].astype(str).replace(num)
#将最后的B需要的数据输出来
B_periods_time1 = B_periods_time1.loc[:, ['date', 'post_id','periods','amount']]

#拆分日期和时间
temp4 = pd.DatetimeIndex(B_periods_time2['date'])
B_periods_time2['date'] = temp4.date
B_periods_time2['time'] = temp4.time
#替换会时间段
B_periods_time2.reset_index(inplace=True, drop=True)
B_periods_time2['periods'] = B_periods_time2['time'].astype(str).replace(num)
#将最后的B需要的数据输出来
B_periods_time2 = B_periods_time2.loc[:, ['date', 'post_id','periods','amount']]

In [13]:
#取特定时间段
A_periods_time1 = A_periods_time1.loc[A_periods_time1['periods'].astype(str).isin(['19','21','23','25','27','29','31','33','35','37'])]
A_periods_time2 = A_periods_time2.loc[A_periods_time2['periods'].astype(str).isin(['18','20','22','24','26','28','30','32','34','36'])]
B_periods_time1 = B_periods_time1.loc[B_periods_time1['periods'].astype(str).isin(['19','21','23','25','27','29','31','33','35','37'])]
B_periods_time2 = B_periods_time2.loc[B_periods_time2['periods'].astype(str).isin(['18','20','22','24','26','28','30','32','34','36'])]

In [14]:
#日期换格式
A_periods_time1.reset_index(inplace=True, drop=True)
A_periods_time1['date'] = A_periods_time1['date'].astype(str).str.replace('-','/')
A_periods_time1['date'] = A_periods_time1['date'].astype(str).str.replace('/01','/1')
A_periods_time1['date'] = A_periods_time1['date'].astype(str).str.replace('/02','/2')
A_periods_time1['date'] = A_periods_time1['date'].astype(str).str.replace('/03','/3')
A_periods_time1['date'] = A_periods_time1['date'].astype(str).str.replace('/04','/4')
A_periods_time1['date'] = A_periods_time1['date'].astype(str).str.replace('/05','/5')
A_periods_time1['date'] = A_periods_time1['date'].astype(str).str.replace('/06','/6')
A_periods_time1['date'] = A_periods_time1['date'].astype(str).str.replace('/07','/7')
A_periods_time1['date'] = A_periods_time1['date'].astype(str).str.replace('/08','/8')
A_periods_time1['date'] = A_periods_time1['date'].astype(str).str.replace('/09','/9')
A_periods_time2.reset_index(inplace=True, drop=True)
A_periods_time2['date'] = A_periods_time2['date'].astype(str).str.replace('-','/')
A_periods_time2['date'] = A_periods_time2['date'].astype(str).str.replace('/01','/1')
A_periods_time2['date'] = A_periods_time2['date'].astype(str).str.replace('/02','/2')
A_periods_time2['date'] = A_periods_time2['date'].astype(str).str.replace('/03','/3')
A_periods_time2['date'] = A_periods_time2['date'].astype(str).str.replace('/04','/4')
A_periods_time2['date'] = A_periods_time2['date'].astype(str).str.replace('/05','/5')
A_periods_time2['date'] = A_periods_time2['date'].astype(str).str.replace('/06','/6')
A_periods_time2['date'] = A_periods_time2['date'].astype(str).str.replace('/07','/7')
A_periods_time2['date'] = A_periods_time2['date'].astype(str).str.replace('/08','/8')
A_periods_time2['date'] = A_periods_time2['date'].astype(str).str.replace('/09','/9')
#日期换格式
B_periods_time1.reset_index(inplace=True, drop=True)
B_periods_time1['date'] = B_periods_time1['date'].astype(str).str.replace('-','/')
B_periods_time1['date'] = B_periods_time1['date'].astype(str).str.replace('/01','/1')
B_periods_time1['date'] = B_periods_time1['date'].astype(str).str.replace('/02','/2')
B_periods_time1['date'] = B_periods_time1['date'].astype(str).str.replace('/03','/3')
B_periods_time1['date'] = B_periods_time1['date'].astype(str).str.replace('/04','/4')
B_periods_time1['date'] = B_periods_time1['date'].astype(str).str.replace('/05','/5')
B_periods_time1['date'] = B_periods_time1['date'].astype(str).str.replace('/06','/6')
B_periods_time1['date'] = B_periods_time1['date'].astype(str).str.replace('/07','/7')
B_periods_time1['date'] = B_periods_time1['date'].astype(str).str.replace('/08','/8')
B_periods_time1['date'] = B_periods_time1['date'].astype(str).str.replace('/09','/9')
B_periods_time2.reset_index(inplace=True, drop=True)
B_periods_time2['date'] = B_periods_time2['date'].astype(str).str.replace('-','/')
B_periods_time2['date'] = B_periods_time2['date'].astype(str).str.replace('/01','/1')
B_periods_time2['date'] = B_periods_time2['date'].astype(str).str.replace('/02','/2')
B_periods_time2['date'] = B_periods_time2['date'].astype(str).str.replace('/03','/3')
B_periods_time2['date'] = B_periods_time2['date'].astype(str).str.replace('/04','/4')
B_periods_time2['date'] = B_periods_time2['date'].astype(str).str.replace('/05','/5')
B_periods_time2['date'] = B_periods_time2['date'].astype(str).str.replace('/06','/6')
B_periods_time2['date'] = B_periods_time2['date'].astype(str).str.replace('/07','/7')
B_periods_time2['date'] = B_periods_time2['date'].astype(str).str.replace('/08','/8')
B_periods_time2['date'] = B_periods_time2['date'].astype(str).str.replace('/09','/9')

In [15]:
#对B筛选只要工作日的数据
work = wkd_v1.loc[(wkd_v1['wkd_typ_cd']=='WN')|(wkd_v1['wkd_typ_cd']=='WS')]
#string to datetime才能选范围
work['date'] = pd.to_datetime(work['date'], format='%Y-%m-%d %H:%M:%S')
work = work.loc[(work['date']>='2020/11/1')&(work['date']<='2020/11/30')]
#datetime to string 才能merge
work['date'] = work['date'].apply(lambda x: x.strftime('%Y-%m-%d'))
work.reset_index(inplace=True, drop=True)
work['date'] = work['date'].astype(str).str.replace('-','/')
work['date'] = work['date'].astype(str).str.replace('/01','/1')
work['date'] = work['date'].astype(str).str.replace('/02','/2')
work['date'] = work['date'].astype(str).str.replace('/03','/3')
work['date'] = work['date'].astype(str).str.replace('/04','/4')
work['date'] = work['date'].astype(str).str.replace('/05','/5')
work['date'] = work['date'].astype(str).str.replace('/06','/6')
work['date'] = work['date'].astype(str).str.replace('/07','/7')
work['date'] = work['date'].astype(str).str.replace('/08','/8')
work['date'] = work['date'].astype(str).str.replace('/09','/9')
B_periods_time1 = pd.merge(B_periods_time1, work, how='left', on=['date'])
B_periods_time1 = B_periods_time1.fillna(0)
B_periods_time1 = B_periods_time1.loc[B_periods_time1['wkd_typ_cd'] != 0]
B_periods_time1 = B_periods_time1.loc[:,['date','post_id','periods','amount']]
B_periods_time2 = pd.merge(B_periods_time2, work, how='left', on=['date'])
B_periods_time2 = B_periods_time2.fillna(0)
B_periods_time2= B_periods_time2.loc[B_periods_time2['wkd_typ_cd'] != 0]
B_periods_time2 = B_periods_time2.loc[:,['date','post_id','periods','amount']]

# 保存每半小时粒度预测结果

In [16]:
#合并Atime1
A_periods_predit = pd.merge(test_v2_periods,A_periods_time1,how='left',on=['date','post_id','periods'])
A_periods_predit = A_periods_predit.fillna(0)
A_periods_predit['amount'] = A_periods_predit['amount_x'] + A_periods_predit['amount_y']
A_periods_predit = A_periods_predit.loc[:,['date','post_id','periods','amount']]
A_periods_predit['amount'] = A_periods_predit['amount'].astype(int)
#合并Atime2
A_periods_predit = pd.merge(A_periods_predit,A_periods_time2,how='left',on=['date','post_id','periods'])
A_periods_predit = A_periods_predit.fillna(0)
A_periods_predit['amount'] = A_periods_predit['amount_x'] + A_periods_predit['amount_y']
A_periods_predit = A_periods_predit.loc[:,['date','post_id','periods','amount']]
A_periods_predit['amount'] = A_periods_predit['amount'].astype(int)
#合并Btime1
result = pd.merge(A_periods_predit,B_periods_time1,how='left',on=['date','post_id','periods'])
result = result.fillna(0)
result['amount'] = result['amount_x'] + result['amount_y']
result = result.loc[:,['date','post_id','periods','amount']]
result['amount'] = result['amount'].astype(int)
#合并Btime2
result = pd.merge(result,B_periods_time2,how='left',on=['date','post_id','periods'])
result = result.fillna(0)
result['amount'] = result['amount_x'] + result['amount_y']
result = result.loc[:,['date','post_id','periods','amount']]
result['amount'] = result['amount'].astype(int)

In [17]:
result.to_csv('./result/test_B_periods_20210511_1.txt', index=False, header=False, sep=',')
result

,date,post_id,periods,amount
0,2020/12/1,A,1,0
1,2020/12/1,A,2,0
2,2020/12/1,A,3,0
3,2020/12/1,A,4,0
4,2020/12/1,A,5,0
5,2020/12/1,A,6,0
6,2020/12/1,A,7,0
7,2020/12/1,A,8,0
8,2020/12/1,A,9,0
9,2020/12/1,A,10,0
